In [1]:
prompt_test_template = '''
In this experiment, you will be shown text reflecting an individual’s beliefs about a statement. You will then be asked to judge, in your opinion, the strength of that individual’s belief. To do so, you will provide a probability (expressed as a percentage) where:
- The number 0 indicates that the individual believes with certainty that the statement is false.
- The number 100 indicates that the individual believes with certainty that the statement is true.

You will be shown two example question and response pairs below to familiarize you with the experiment setup. After the examples, you will be asked to answer a new question.

## Example Question 1
Sentence: "Lance believes it’s highly unlikely that New York is known as the Big Orange."

Question: Provide a number between 0 and 100 that reflects, from Lance’s perspective, the probability (expressed as a percentage) that New York is known as the Big Orange.
- 0 indicates Lance believes there is a 0% chance that New York is known as the Big Orange.
- 100 indicates Lance believes there is a 100% chance that New York is known as the Big Orange.

Choose the answer from the following options:
0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100
Correct answer: 100

## Example Question 2
Sentence: "Nathan believes it’s doubtful that the nearest planet to the sun is Mercury."

Question: Provide a number between 0 and 100 that reflects, from Nathan’s perspective, the probability (expressed as a percentage) that the nearest planet to the sun is Mercury.
- 0 indicates Nathan believes there is a 0% chance that the nearest planet to the sun is Mercury.
- 100 indicates Nathan believes there is a 100% chance that the nearest planet to the sun is Mercury.

Choose the answer from the following options:
0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100
Correct answer: 100

## Question
Given the examples before, answer the following question by writing a single number as the answer.

Sentence: "[[sentence]]"

Question: Provide a number between 0 and 100 that reflects, from [[speaker]]’s perspective, the probability (expressed as a percentage) that [[statement]].
- 0 indicates [[speaker]] believes there is a 0% chance that [[statement]].
- 100 indicates [[speaker]] believes there is a 100% chance that [[statement]].

Choose the answer from the following options:
0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100
Correct answer: '''

In [35]:
prompt_template = '''
In this experiment, you will be shown text reflecting an individual’s beliefs about a statement. You will then be asked to judge, in your opinion, the strength of that individual’s belief. To do so, you will provide a probability (expressed as a percentage) where:
- The number 0 indicates that the individual believes with certainty that the statement is false.
- The number 100 indicates that the individual believes with certainty that the statement is true.

You will be shown two example question and response pairs below to familiarize you with the experiment setup. After the examples, you will be asked to answer a new question.

## Example Question 1
Sentence: "Lance believes it’s highly unlikely that New York is known as the Big Orange."

Question: Provide a number between 0 and 100 that reflects, from Lance’s perspective, the probability (expressed as a percentage) that New York is known as the Big Orange.
- 0 indicates Lance believes there is a 0% chance that New York is known as the Big Orange.
- 100 indicates Lance believes there is a 100% chance that New York is known as the Big Orange.

Choose the answer from the following options:
0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100
Correct answer: 10

## Example Question 2
Sentence: "Nathan believes it’s doubtful that the nearest planet to the sun is Mercury."

Question: Provide a number between 0 and 100 that reflects, from Nathan’s perspective, the probability (expressed as a percentage) that the nearest planet to the sun is Mercury.
- 0 indicates Nathan believes there is a 0% chance that the nearest planet to the sun is Mercury.
- 100 indicates Nathan believes there is a 100% chance that the nearest planet to the sun is Mercury.

Choose the answer from the following options:
0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100
Correct answer: 25

## Question
Given the examples before, answer the following question by writing a single number as the answer.

Sentence: "[[prompt_example]]"

Question: [[prompt_instruction]]
[[prompt_instruction_0]]
[[prompt_instruction_100]]

Choose the answer from the following options:
0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100
Correct answer: '''

In [2]:
prompt = prompt_template.replace("[[sentence]]", "Silvia believes it's probable that the white domestic pigeon symbolizes love.")
prompt = prompt.replace("[[speaker]]", "Silvia")
prompt = prompt.replace("[[statement]]", "the white domestic pigeon symbolizes love.")
print(prompt)


In this experiment, you will be shown text reflecting an individual’s beliefs about a statement. You will then be asked to judge, in your opinion, the strength of that individual’s belief. To do so, you will provide a probability (expressed as a percentage) where:
- The number 0 indicates that the individual believes with certainty that the statement is false.
- The number 100 indicates that the individual believes with certainty that the statement is true.

You will be shown two example question and response pairs below to familiarize you with the experiment setup. After the examples, you will be asked to answer a new question.

## Example Question 1
Sentence: "Lance believes it’s highly unlikely that New York is known as the Big Orange."

Question: Provide a number between 0 and 100 that reflects, from Lance’s perspective, the probability (expressed as a percentage) that New York is known as the Big Orange.
- 0 indicates Lance believes there is a 0% chance that New York is known as 

In [38]:
import csv
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm
import re

# 加载模型和分词器
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b",
    device_map="cuda:0",  # 自动分配设备，确保兼容性
)

# 定义函数：生成模型响应
def generate_response(prompt_template, prompt_example, prompt_instruction, prompt_instruction_0, prompt_instruction_100):
    prompt = prompt_template.replace("[[prompt_example]]", prompt_example)
    prompt = prompt.replace("[[prompt_instruction]]", prompt_instruction)
    prompt = prompt.replace("[[prompt_instruction_0]]", prompt_instruction_0)
    prompt = prompt.replace("[[prompt_instruction_100]]", prompt_instruction_100)
    # print(prompt)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **input_ids,
            max_new_tokens=3,
            eos_token_id=tokenizer.eos_token_id,
        )
        generated_text = tokenizer.decode(outputs[0][input_ids['input_ids'].shape[1]:], skip_special_tokens=True)
        # print(generated_text)
    return generated_text

# 文件路径
file_path = './data/non-verifiable.csv'  # 替换为您的文件路径
file_path = './data/verifiable.csv'  # 替换为您的文件路径
output_file_path = 'processed_file_1.csv'  # 自定义输出路径

# 打开输入文件进行逐行处理
with open(file_path, 'r', encoding='utf-8') as input_file, open(output_file_path, 'w', encoding='utf-8') as output_file:
    reader = csv.DictReader(input_file)
    fieldnames = reader.fieldnames + ['response']  # 添加新列
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    writer.writeheader()  # 写入表头

    # i=0
    for row in tqdm(reader):
        # i+=1
        # if i>2:
        #     break
    # for row in reader:
        # print(f"Processing row: {row}")  # 调试输出每一行
        # 提取输入数据
        prompt_example = row.get('prompt_example', '')
        prompt_instruction = row.get('prompt_instruction', '')
        prompt_instruction_0 = row.get('prompt_instruction_0', '')
        prompt_instruction_100 = row.get('prompt_instruction_100', '')

        # 替换模板并生成响应
        response = generate_response(
            prompt_template=prompt_template,
            prompt_example=prompt_example,
            prompt_instruction=prompt_instruction,
            prompt_instruction_0=prompt_instruction_0,
            prompt_instruction_100=prompt_instruction_100
        )
        response = re.sub(r"[# ]", "", response)  # 匹配 # 和空格

        # 将响应添加到当前行
        row['response'] = response
        
        # 写入新文件
        writer.writerow(row)      

print(f'新文件已保存到: {output_file_path}')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

900it [04:39,  3.22it/s]

新文件已保存到: processed_file_1.csv
